In [1]:
import math
def entropy_func(c,n):
  return -(c*1.0/n)*math.log(c*1.0/n,2)

def entropy_cal(c1,c2):
  if c1 == 0 or c2 == 0:
    return 0
  return entropy_func(c1,c1+c2) + entropy_func(c2,c1+c2)

def entropy_of_one_division(division): 
    s = 0
    n = len(division)
    classes = set(division)
    for c in classes:  
        n_c = sum(division==c)
        e = n_c*1.0/n * entropy_cal(sum(division==c), sum(division!=c)) 
        s += e
    return s, n

def get_entropy(y_predict, y_real):
    if len(y_predict) != len(y_real):
        print('They have to be the same length')
        return None
    n = len(y_real)
    s_true, n_true = entropy_of_one_division(y_real[y_predict])
    s_false, n_false = entropy_of_one_division(y_real[~y_predict])
    s = n_true*1.0/n * s_true + n_false*1.0/n * s_false 
    return s

import numpy as np
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':y[0]}
        elif depth >= self.max_depth:
            return None
        else: 
            col, cutoff, entropy = self.find_best_split_of_all(x, y)    
            y_left = y[x[:, col] < cutoff]
            y_right = y[x[:, col] >= cutoff]
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))}
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            if entropy == 0:    
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    def find_best_split(self, col, y):
        """
        col: the column we split on
        y: target var
        """
        min_entropy = 10
        n = len(y)
        for value in set(col):
            y_predict = col < value
            my_entropy = get_entropy(y_predict, y)
            if my_entropy <= min_entropy:
                min_entropy = my_entropy
                cutoff = value
        return min_entropy, cutoff
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results
    
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')

from sklearn.datasets import load_iris
from pprint import pprint

iris = load_iris()

x = iris.data
y = iris.target

clf = DecisionTreeClassifier(max_depth=7)
m = clf.fit(x, y)

pprint(m)

import pandas as pd
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
data1

x_test = [[5.5,3.5,1.4,0.3]]
clf.predict(x_test)

{'col': 'petal width (cm)',
 'cutoff': 1.0,
 'index_col': 3,
 'left': {'val': 0},
 'right': {'col': 'petal width (cm)',
           'cutoff': 1.8,
           'index_col': 3,
           'left': {'col': 'petal length (cm)',
                    'cutoff': 5.0,
                    'index_col': 2,
                    'left': {'col': 'petal width (cm)',
                             'cutoff': 1.7,
                             'index_col': 3,
                             'left': {'val': 1},
                             'right': {'val': 2},
                             'val': 1.0},
                    'right': {'col': 'petal width (cm)',
                              'cutoff': 1.6,
                              'index_col': 3,
                              'left': {'val': 2},
                              'right': {'col': 'sepal length (cm)',
                                        'cutoff': 7.2,
                                        'index_col': 0,
                                        'left

array([0])